<div>
<img src='http://www-scf.usc.edu/~ghasemig/images/sharif.png' alt="SUT logo" width=220 height=220 align=left class="saturate">

<br>
<font face="Times New Roman">
<div dir=ltr align=center> 
<!-- <font color=0F5298 size=7> -->
<font color=0F5298 size=6>
    Introduction to Machine Learning <br> <br>
<!-- <font color=2565AE size=5> -->
<font size=5>
    Computer Engineering Department <br>
    Spring 2023 <br> <br>
<font color=606060 size=5>
    Homework 4: Practical - Naive Bayes <br> <br>
<font color=686880 size=4>
    TAs: Alireza Farashah - Arman Malekzadeh - Ali Salesi
    
____

### Full Name : Erfan Sadraiye
### Student Number : 99101835
### Colab Link: https://colab.research.google.com/drive/1PrQ0Xc7otjXLcIKlAVzVr2I3Qogc69Nw?usp=sharing
___

Note 1: In this assignment, we are trying to simulate the functionality of something called "CountVectorizer" which is used in natural language processing. You are advised to take a look at this link before beginning to answer the questions:

https://www.geeksforgeeks.org/using-countvectorizer-to-extracting-features-from-text/

Note 2: One or two TA sessions will be held related to this assignment to make sure you get familiarized with this topic. Therefore, keep calm and write code!

# Download the Dataset

In [1]:
!wget -O /content/dataset.zip "https://www.dropbox.com/s/yf195wl0sp3term/dataset-train.zip?dl=1"

--2023-05-05 12:59:21--  https://www.dropbox.com/s/yf195wl0sp3term/dataset-train.zip?dl=1
Resolving www.dropbox.com (www.dropbox.com)... 162.125.6.18, 2620:100:601f:18::a27d:912
Connecting to www.dropbox.com (www.dropbox.com)|162.125.6.18|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: /s/dl/yf195wl0sp3term/dataset-train.zip [following]
--2023-05-05 12:59:21--  https://www.dropbox.com/s/dl/yf195wl0sp3term/dataset-train.zip
Reusing existing connection to www.dropbox.com:443.
HTTP request sent, awaiting response... 302 Found
Location: https://uc66a635d3f40ced0ec19aa4cdac.dl.dropboxusercontent.com/cd/0/get/B7fJNnMONy1j4gIMK0vxrw8nkBEqgWimqAlvDq0wiFqjopBu6kVGtUrk5VX9gvtyO8Ki7OGAwmud00KQdKYhl-Jej4YjOsIPPexytU9sLoCxsmiSLecdnlfNsjfeZZGn0HXV6j8p1Ldpls8xXXs_drEPetgCurgi_slC7T6jLUBtchzp9Th4k9-GcOAS7GdZCG8/file?dl=1# [following]
--2023-05-05 12:59:21--  https://uc66a635d3f40ced0ec19aa4cdac.dl.dropboxusercontent.com/cd/0/get/B7fJNnMONy1j4gIMK0vxrw8nkBEqgWimqAlvDq0wiF

In [2]:
!mkdir /content/dataset/
!unzip '/content/dataset.zip' -d /content/dataset/

Archive:  /content/dataset.zip
  inflating: /content/dataset/content/news-train.csv  


# Preprocessing

In [3]:
import pandas as pd
import numpy as np
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.naive_bayes import MultinomialNB
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report

Read the csv file and load it as a dataframe.

In [4]:
df = pd.read_csv('dataset/content/news-train.csv')

In [5]:
df.head(5)

,topic,text
0,sports,به گزارش ورزش سه تیم فوتبال پدیده در روزی که ا...
1,politics,پاسخ مثبت به انتقادات شورای نگهبان البته # قبل...
2,politics,# دهم در پاسخ به سوالی درباره اینکه آیا سفر وی...
3,economics,کنفرانس # بودن بازار خودرو و تاثیر آن بر اقتصا...
4,economics,به گزارش ایسنا صحبت با برخی # # آن است که امرو...


# Dealing with Class Imbalance

For each "topic", count and display the number of news belonging to it. (5 points)

In [6]:
df['topic'].value_counts()

sports       123105
economics     49600
cultural      13359
politics      11297
Name: topic, dtype: int64

Balance the dataset in a way that all of the topics get associated with the same number of news. For this purpose, find the topic for which the minimum number of news exists. Then, use downsampling to lower the number of news associated with the other topics.<br>
Finally, save the result as a new dataframe. (15 points)

In [7]:
min_topic = df['topic'].value_counts().idxmin()
min_topic_count = df['topic'].value_counts().min()
balance_df = df.groupby('topic').apply(lambda x: x.sample(min_topic_count,random_state=42)).reset_index(drop=True)
balance_df['topic'].value_counts()

cultural     11297
economics    11297
politics     11297
sports       11297
Name: topic, dtype: int64

# Preparation and Feature Extraction

Split the dataframe to two parts for training (80%) and testing (20%). (5 points)

In [8]:
X = balance_df['text']
y = balance_df['topic']
X_train, X_test, Y_train, Y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In this part, we will extract features from the textual data using a very basic method.

Consider the following sentence:
```
Sometimes a dog can run faster than a cat.
```
In the above sentence, the "tokens" are:

```
"sometimes", "a", "dog", "can", "run", "faster", "than", "a", "cat"
```

Find the tokens of each news article in the "training dataframe" by splitting it based on the occurence of the space character (5 points). For instance, the following sentence:
```
او به میدان رفت
```
gets converted to these tokens:
```
["او","به","میدان","رفت"]
```

In [9]:
X_train_tokens = X_train.apply(lambda x: x.split(' '))
X_train_tokens.head(5)

20371    [وزیر, راه, و, شهرسازی, یکی, از, مشکلات, اقتصا...
13177    [عضو, هیات‌مدیره, اتحادیه, صادرکنندگان, نفت, گ...
4194     [به, گزارش, ایسنا, مراسم, رونمایی, و, جشن, امض...
11348    [قیمت, دلار, طی, روزهای, اخیر, روند, صعودی, به...
4497     [مهلت, ارسال, آثار, در, هشتمین, جشنواره, بازی‌...
Name: text, dtype: object

Count the tokens and list the top 5 frequent ones (5 points)

In [10]:
tokens = []
for i in range(len(X_train_tokens)):
    tokens += X_train_tokens.iloc[i]
tokens = pd.Series(tokens)
print(tokens.size)
tokens.value_counts().head(5)


20004306


#     1621658
و      877526
در     741868
\n     589125
به     581027
dtype: int64

Consider the following sentence:
```
تیم فوتبال بارسلونا امشب با بایرن مونیخ دیدار می‌کند.
```
In the above sentence, we do not need the word `با` to determine that the topic is "sports". This word doesn't have any impact on the category this sentence belongs to. 
This word and similar ones are called "stopwords". A list of stopwords for the Persian language can be found in this text file:
https://github.com/ziaa/Persian-stopwords-collection/raw/master/Stopwords/Savoy/persianST.txt

Use it to remove all tokens that are actually stopwords.
Also, remove the `#` token. (10 points)

In [11]:
!wget -O /content/stopwords.txt https://github.com/ziaa/Persian-stopwords-collection/raw/master/Stopwords/Savoy/persianST.txt

--2023-05-05 13:00:01--  https://github.com/ziaa/Persian-stopwords-collection/raw/master/Stopwords/Savoy/persianST.txt
Resolving github.com (github.com)... 140.82.112.4
Connecting to github.com (github.com)|140.82.112.4|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://raw.githubusercontent.com/ziaa/Persian-stopwords-collection/master/Stopwords/Savoy/persianST.txt [following]
--2023-05-05 13:00:01--  https://raw.githubusercontent.com/ziaa/Persian-stopwords-collection/master/Stopwords/Savoy/persianST.txt
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 2942 (2.9K) [text/plain]
Saving to: ‘/content/stopwords.txt’

/content/stopwords. 100%[===================>]   2.87K  --.-KB/s    in 0s      

2023-05-05 13:00:01 (24.5 MB/s)

In [12]:
stopwords = pd.read_csv('stopwords.txt', header=None)[0].tolist()
removing_tokens = ['#', '\n'] + stopwords
tokens = tokens[~tokens.isin(removing_tokens)]
tokens.value_counts().head(5)

این      410168
برای     150935
ایران    101417
کرد       97751
کشور      83365
dtype: int64

Make a list of the remaining unique tokens (5 points)

In [13]:
unique_tokens = tokens.unique().tolist()

Count the number of the occurences of each token in each document. This way, you can make a numpy array to represent each document. Call this new array `x_train`. (10 points)

Example:

Unique tokens in all documents: 
```
0: hen
1: sometimes
2: a
3: the
4: dog
5: runs
6: faster
7: than
8: cat
9: bird
```

Current Document: `sometimes a dog runs faster than a cat`

The representation of the document:

```
[0, 1, 2, 0, 1, 1, 1, 1, 1, 0]
```

The meaning of this representation:

```
0: the document doesn't contain "hen"
1: the document contains 1 "sometimes"
2: the document contains 2 "a"
0: the document doesn't contain "the"
1: the document contains 1 "dog"
1: the document contains 1 "runs"
1: the document contains 1 "faster"
1: the document contains 1 "than"
1: the document contains 1 "cat"
0: the document doesn't contain "bird"
``` 

In [14]:
vectorizer = CountVectorizer(vocabulary=unique_tokens)
x_train = vectorizer.fit_transform(X_train)
x_train = x_train.toarray()

Make another numpy array by converting the topics associated with the training news to numbers. For instance, if the topics are: `sports`, `economics`, `politics`, and `cultural`, convert them to `0` to `3`. Call this new array `y_train`. (5 points)

In [15]:
encoder = LabelEncoder()
y_train = encoder.fit_transform(Y_train)

Using the unique tokens you found in the training news, represent the testing news as another numpy array called `x_test`. (5 points)

Note: Do not forget to remove stopwords and the `#` token.

In [16]:
x_test = vectorizer.transform(X_test)
x_test = x_test.toarray()

Convert the topics associated with the testing news to their equivalent numbers (as before), and save the result as a numpy array called `y_test`. (5 points)

In [17]:
y_test = encoder.transform(Y_test)

# Model Training and Evaluation

## Using sklearn (10 points)

Train a Multinomial Naive Bayes Model on the training news (`x_train` and `y_train`) using `sklearn`.

In [18]:
sk_model = MultinomialNB()
sk_model.fit(x_train, y_train)

MultinomialNB()

Get the predictions of the model for the testing news (`x_test`).

In [19]:
y_pred_sk = sk_model.predict(x_test)

Print the classification report containing "precision", "recall", and "f1-score" for each class, and their averages. (you can use `sklearn` for this part)

In [20]:
print(classification_report(y_test, y_pred_sk))

              precision    recall  f1-score   support

           0       0.94      0.96      0.95      2253
           1       0.92      0.94      0.93      2262
           2       0.95      0.91      0.93      2268
           3       0.99      0.99      0.99      2255

    accuracy                           0.95      9038
   macro avg       0.95      0.95      0.95      9038
weighted avg       0.95      0.95      0.95      9038



## Using your own code (15 points)

In [21]:
class Naive_Bayes:
    """
    Fits it on data, then uses predict to get results.
    """

    def __init__(self,alpha=1):
        self.log_cond_prob = None
        self.cond_prob = None
        self.prior = None
        self.alpha = alpha
        self.classes_ = None

    def fit(self, X, y):
        """Fit the training data
        Parameters
        ----------
        x : array-like, shape = [n_samples, n_features]
            Training samples
        y : array-like, shape = [n_samples, n_target_values]
            Target values
        Returns
        -------
        self : object
        """

        n_samples, n_features = X.shape
        self.classes_ = np.unique(y)
        n_classes = len(self.classes_)
        self.prior = np.zeros(n_classes)
        self.cond_prob = np.zeros((n_classes, n_features))
        for i, c in enumerate(self.classes_):
            self.prior[i] = np.sum(c == y) / n_samples
            X_c = X[c == y]
            count_of_words_in_class = X_c.sum(axis=0)
            sum_of_words_in_class = count_of_words_in_class.sum()
            self.cond_prob[i] = (count_of_words_in_class + self.alpha) / (
                        sum_of_words_in_class + self.alpha * n_features)
        self.log_cond_prob = np.log(self.cond_prob)
        return self

    def predict(self, X):
        """ Predicts the value after the model has been trained.
        Parameters
        ----------
        x : array-like, shape = [n_samples, n_features]
            Test samples
        Returns
        -------
        Predicted value
        """
        y_pred = None
        prob = np.dot(X, self.log_cond_prob.T) + np.log(self.prior)
        y_pred = self.classes_[np.argmax(prob, axis=1)]
        return y_pred

Train your own Multinomial Naive Bayes Model on the training news (`x_train` and `y_train`).

In [22]:
model = Naive_Bayes()
model.fit(x_train, y_train)

Get the predictions of your model for the testing news (`x_test`).

In [23]:
y_pred = model.predict(x_test)

Print the classification report containing "precision", "recall", and "f1-score" for each class, and their averages. (you can use `sklearn` for this part)

In [24]:
print(classification_report(y_test, y_pred))

              precision    recall  f1-score   support

           0       0.94      0.96      0.95      2253
           1       0.92      0.94      0.93      2262
           2       0.95      0.91      0.93      2268
           3       0.99      0.99      0.99      2255

    accuracy                           0.95      9038
   macro avg       0.95      0.95      0.95      9038
weighted avg       0.95      0.95      0.95      9038



# Kaggle

Competition Link: https://www.kaggle.com/t/88f1b6e251e34575b2e4cb4b91aed0ef

In [25]:
!kaggle competitions download -c sharif-ml-1401-c4

 97% 169M/175M [00:01<00:00, 94.5MB/s]
100% 175M/175M [00:01<00:00, 101MB/s] 


In [26]:
!mkdir /content/kaggle/
!unzip 'sharif-ml-1401-c4.zip' -d /content/kaggle/

Archive:  sharif-ml-1401-c4.zip
  inflating: /content/kaggle/assignment4-test-data-sample-submission.csv  
  inflating: /content/kaggle/assignment4-test-data.csv  
  inflating: /content/kaggle/assignment4-training-data.csv  


In [27]:
x_test_kaggle_df = pd.read_csv('/content/kaggle/assignment4-test-data.csv')
x_test_kaggle_df_text = x_test_kaggle_df['text']
x_test_kaggle = vectorizer.transform(x_test_kaggle_df_text).toarray()

In [32]:
def kaggle_label_encoder(prediction_list):
    class_map = {'politics': 1, 'economics': 2, 'sports': 3, 'cultural': 4}
    return np.array([class_map[item] for item in prediction_list])
       
y_pred_kaggle = model.predict(x_test_kaggle)
y_pred_kaggle_label = encoder.inverse_transform(y_pred_kaggle)
y_pred_kaggle_upload = kaggle_label_encoder(y_pred_kaggle_label)

In [39]:
y_pred_kaggle_df = pd.DataFrame(y_pred_kaggle_upload, columns=['topic'])
y_pred_kaggle_df['ID'] = np.arange(1, y_pred_kaggle_upload.shape[0]+1)
y_pred_kaggle_df.to_csv('submission.csv', index=False)

In [41]:
!kaggle competitions submit -c sharif-ml-1401-c4 -f submission.csv -m "Message"

100% 2.83k/2.83k [00:00<00:00, 5.12kB/s]
Successfully submitted to News Classification